In [2]:
!git clone https://github.itap.purdue.edu/guo-research-group/Ray-Optics

Cloning into 'Ray-Optics'...
remote: Enumerating objects: 508, done.
remote: Total 508 (delta 0), reused 0 (delta 0), pack-reused 508
Receiving objects: 100% (508/508), 1.59 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (168/168), done.


In [3]:
%cd /content/Ray-Optics
!python setup.py develop

/content/Ray-Optics
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other stan

In [4]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
  

In [1]:
from rayoptics import *

In [2]:
from rayoptics.environment import *
from rayoptics.mpl.interactivediagram import InteractiveDiagram


In [3]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [4]:
from rayoptics.environment import *
import random
from copy import deepcopy
from scipy.optimize import minimize
import numpy as np

In [5]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.numpy import sqrt, copysign, sin
jax.config.update("jax_enable_x64", True)

In [6]:
from jax import lax

In [7]:
isdark = False

In [8]:
from matplotlib.ticker import AutoMinorLocator
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from scipy.spatial import *

def visualize_rays(sm=None, max_angle=None, wv= 400.5618, x_offsets=[0], y_offsets=[0], num_rays= 1, color = 'green'):
    """
    This function visualizes rays in an optical system by creating a 3D scatter plot for each ray.

    Parameters:
    sm (System): The optical system to visualize.
    max_angle (float): The maximum incident angle, in radians.
    wv (float): The wavelength of the rays.
    x_offsets (list): The x-coordinates of the starting points of the rays.
    y_offsets (list): The y-coordinates of the starting points of the rays.
    z_offsets (list): The z-coordinates of the starting points of the rays.
    num_rays (int): The number of rays to visualize from each starting point. The total number of rays at each starting point is num_rays^2.

    Returns:
    data (list): A list of plotly Scatter3d objects, one for each ray in the system.

    The function works by iterating over each starting point and each angle offset, creating a ray at each combination.
    It then traces the path of each ray through the system, collecting the coordinates of the ray at each point.
    These coordinates are then used to create a 3D scatter plot of the ray, which is added to the data list.
    The function also visualizes the intersection point of each ray with the photosensor, if it is within the bounds of the system.
    """

    if max_angle is not None:
         # Tan Angle Offsets
        angle_offsets = np.linspace(-max_angle, max_angle, num_rays)
    else:
        angle_offsets = [0]

    #Stores Figure data
    data = []
    # Creates each starting coordinate with the y_offset
    # Can add extra for loops if want to have other axis
    idx = 1
    for x_offset in x_offsets:
        for y_offset in y_offsets:
          for ray in angle_offsets: # Creates each tan angle offset
            for ray_angle in angle_offsets:  # Creates each inc angle
                  #Sets incident angle and finds ray direction
                  inc_angle = ray_angle
                  si = np.sin(inc_angle)
                  cs = np.cos(inc_angle)
                  tn = np.tan(ray)

                  st_coord = np.array([x_offset * 0.1 ,y_offset * 0.1, 0]) # Ray start Co-ord (Add offsets here to alter starting position)

                  st_dir = np.array([tn,si,cs]) # Ray starting direction

                  output = trace(sm, st_coord, st_dir, wvl=wv) #Traces Ray

                  #Collects coordinates of traced ray for visualization
                  pt_photosensor = output[0][-1][0]
                  if np.abs(pt_photosensor[1]) <= 50:
                      x = []
                      y = []
                      z = []
                      z_bias = 0
                      j = 0
                      for pt in output[0][0::]:
                          y.append(pt[0][1])
                          z.append(pt[0][2]+z_bias)
                          x.append(pt[0][0])
                          if j < len(sm.gaps):
                              z_bias += sm.gaps[j].thi
                              j += 1

                      #Visualizes Ray
                      data.append(go.Scatter3d(x=x, y=z, z=y, mode='lines', line=dict(color=color, width=1), opacity=0.5, name =f"Ray {idx}"))

                      #Visualises intersection point of rays
                      data.append(go.Scatter3d(x=x[::], y= z[::], z=y[::], mode = 'markers', marker=dict(color='black', size=2), showlegend = False))

                      idx += 1
    return data

In [31]:
def visualize_lens(sm):
    N = 10                  # Number of dimensions
    z_bias = sm.gaps[0].thi # Z coord of Lens
    sd = 6                  # Grid Size
    data = []               # Lens visualisation data

    #Goes through each surface and collects mesh data
    for i in range(1, sm.get_num_surfaces()-1):
      mesh = [[],[],[]]
      surf = sm.ifcs[i]
      # sd = np.max([surf.surface_od(),sd])
      for xj in np.linspace(-sd, sd, N):
        for yk in np.linspace(-sd,sd, N):
            z = surf.profile.sag(xj,yk) + z_bias
            #if (z_bias +  sm.gaps[i].thi >= z):
            mesh[0].append(xj)
            mesh[1].append(yk)
            mesh[2].append(z)

      z_bias +=  sm.gaps[i].thi


      #Prepare mesh data for Delaunay
      x = mesh[0]
      y = mesh[1]
      z = mesh[2]

      points = np.vstack([x, y]).T

      # Perform Delaunay triangulation
      tri = Delaunay(points)

      # The indices of the triangles are stored in the `simplices` attribute
      i, j, k = tri.simplices.T

      # Adds mesh to plotly data for plotting
      data.append(go.Mesh3d(x=x, y=z, z=y, i=i, j=j, k=k, intensity=x, colorscale='algae', opacity=1))

    return data

In [32]:
# Jose Sasian’s Lens Design OPTI 517 EXAMPLE

# create a new optical model and set up aliases
opm = OpticalModel()
sm  = opm['seq_model']
osp = opm['optical_spec']
pm = opm['parax_model']
em = opm['ele_model']
pt = opm['part_tree']
ar = opm['analysis_results']

# define field specs
osp['pupil'] = PupilSpec(osp, key=['object', 'pupil'], value=12.5)
osp['fov'] = FieldSpec(osp, key=['object', 'angle'], value=20, flds=[0, 0.707, 1], is_relative=True)
osp['wvls'] = WvlSpec([('F', 0.5), (587.5618, 1.0), ('C', 0.5)], ref_wl=1)
wv = 587.5618

# define interface and gap data
opm.radius_mode = True
sm.gaps[0].thi= 15

# add the surfaces
sm.add_surface([23.713, 4.831, 'N-LAK9', 'Schott'])
sm.add_surface([7331.288, 5.86])
sm.add_surface([-24.456, .975, 'N-SF5,Schott'])
sm.set_stop()
sm.add_surface([21.896, 4.822])
sm.add_surface([86.759, 3.127, 'N-LAK9', 'Schott'])
sm.add_surface([-20.4942, 41.2365])

# update the model
opm.update_model()

# set ray parameters
rays = 5
x_offsets = list(np.linspace(-60, 60, rays))
y_offsets = list(np.linspace(-60, 60, rays))
z_offsets = [0] * rays
num_rays = 1

# get ray and lens data
ray_data = visualize_rays(sm, wv=wv, x_offsets=x_offsets, y_offsets=y_offsets, num_rays=num_rays)
lens_data = visualize_lens(sm)

# plot
data = []
data.extend(ray_data)
data.extend(lens_data)
fig = go.Figure(data = data)
fig.update_scenes(aspectmode='data')
fig.show()